# Trabalho final de Sistemas Baseados em Conhecimento
### Tema: Geração de texto com Markov
### Alunos: Caio Lucas da Silva Chacon (20200025769) e Jonas Gabriel Leite de Araújo (20200007608)

### Dado o início do texto em português, gerar automaticamente algo que faça sentido usando cadeias de Markov

In [59]:
from scipy.sparse import dok_matrix
import numpy as np
import re
from random import random 

# Leitura do arquivo e processamento textual para melhor análise

def read_text(filename):
    """"
    Função que lê um arquivo de texto e retorna o conteúdo em minúsculas
    """

    with open(filename, 'r', encoding="utf8") as file:
        content = file.read().lower()

        content = re.sub(r'[\n,\.;:\"\'_\-\t\—]', ' ', content)
        content = re.sub(r'\s+', ' ', content)

    return content

content = read_text('domCasmurro.txt')
print(content)


dom casmurro capítulo primeiro do título uma noite destas vindo da cidade para o engenho novo encontrei no trem da central um rapaz aqui do bairro que eu conheço de vista e de chapéu cumprimentou me sentou se ao pé de mim falou da lua e dos ministros e acabou recitando me versos a viagem era curta e os versos pode ser que não fossem inteiramente maus sucedeu porém que como eu estava cansado fechei os olhos três ou quatro vezes tanto bastou para que ele interrompesse a leitura e metesse os versos no bolso continue disse eu acordando já acabei murmurou ele são muito bonitos vi lhe fazer um gesto para tirá los outra vez do bolso mas não passou do gesto estava amuado no dia seguinte entrou a dizer de mim nomes feios e acabou alcunhando me dom casmurro os vizinhos que não gostam dos meus hábitos reclusos e calados deram curso à alcunha que afinal pegou nem por isso me zanguei contei a anedota aos amigos da cidade e eles por graça chamam me assim alguns em bilhetes dom casmurro domingo vou j

In [60]:
def get_words(corpus):
    """
    Retorna todas as palavras
    """
    corpus_words = corpus.split(' ')
    corpus_words= [word for word in corpus_words if word != '']
    corpus_words # [...'a', 'wyvern', ',', 'two', 'of', 'the', 'thousand'...]
    return corpus_words # 2185920

def get_distinct_words_count(corpus_words):
    """
    Função que retorna a quantidade de palavras distintas e um dicionário com o índice de cada palavra
    """

    distinct_words = list(set(corpus_words))
    word_idx_dict = {word: i for i, word in enumerate(distinct_words)}
    distinct_words_count = len(list(set(corpus_words)))
 
    return distinct_words_count, word_idx_dict # 32663

### Definição da probabilidade inicial

In [83]:
from collections import Counter

corpus_words = get_words(content)

distinct_words = list(set(corpus_words))
distinct_words_count, word_idx_dict = get_distinct_words_count(corpus_words)

counter_corpus = Counter(corpus_words)

# probabilidade de cada palavra dividido pela quantidade de palavras no corpus
distinct_words_prob = {word: counter_corpus[word]/len(corpus_words) for word in distinct_words}

print(distinct_words_count)
print(word_idx_dict)
print(distinct_words_prob)

9129
{'mesuras': 0, 'purgando': 1, 'linha': 2, 'intensas': 3, 'esqueceu': 4, 'descemos': 5, 'vermelhos': 6, 'conservar': 7, 'aponta': 8, 'alarido': 9, 'crendo': 10, 'desengano': 11, 'medicina?': 12, 'seminário?': 13, 'mudo': 14, 'vigésimo': 15, 'estrangeiras': 16, 'recolheu': 17, 'deitar': 18, 'lidava': 19, 'apertava': 20, 'tornar': 21, 'certíssima!': 22, 'encobertas': 23, 'cântico': 24, 'encarnações': 25, 'assustava': 26, 'muitos': 27, 'depressa?': 28, 'nove': 29, 'cumpre': 30, 'até': 31, 'estaria': 32, 'discreta': 33, 'laivo': 34, 'chorando?': 35, 'véspera': 36, 'quiser': 37, 'penetrar': 38, 'alvoroçado': 39, 'álbum': 40, 'abre!': 41, 'consulte': 42, 'vocês': 43, 'excelentíssima': 44, 'erguiam': 45, 'pertencem': 46, 'mulheres': 47, 'fronteiro': 48, 'conversações': 49, 'tirarem': 50, 'estacava': 51, 'esperavam': 52, 'lápis': 53, 'isso': 54, 'espontâneo': 55, '6': 56, 'casmurrice': 57, 'entenderam': 58, 'persuadir': 59, 'sinhô': 60, 'havidas': 61, 'dinheiros': 62, 'auxílio': 63, 'desej

In [84]:
n_grams = 1 # Numero de palavras anteriores para prever a próxima
# Criação de conjuntos de palavras de tamanho n_grams
sets_of_n_words = [ ' '.join(corpus_words[i:i+n_grams]) for i, _ in enumerate(corpus_words[:-n_grams]) ]
len_n_words = len(list(set(sets_of_n_words)))

print(sets_of_n_words[:10])

['dom', 'casmurro', 'capítulo', 'primeiro', 'do', 'título', 'uma', 'noite', 'destas', 'vindo']


### Criação da matriz de transição

In [85]:
# Inicialização de uma matriz esparsa com tamanho len_n_words, distincit_words_count
# para armazenar a contagem de palavras
n_grams_frequency_matrix = dok_matrix((len_n_words, distinct_words_count))

# Criação de um dicionário para mapear os conjuntos de palavras para índices
distinct_sets_of_n_words = list(set(sets_of_n_words))

# Indice pra cada n_gram
n_gram_idx_dict = {word: i for i, word in enumerate(distinct_sets_of_n_words)}

# Preenchimento da matriz com as contagens
for i, word in enumerate(sets_of_n_words[:-n_grams]): # Percorre todos os bigramas
    # Linha da matriz
    n_gram_idx = n_gram_idx_dict[word] # Indice do n_gram
    # Coluna da matriz
    next_word_idx = word_idx_dict[corpus_words[i+n_grams]] # Palavra que aparece logo após o n_grama
    
    # Preenchimento da matriz com a quantidade de vezes que o bigrama ocorre
    n_grams_frequency_matrix[n_gram_idx, next_word_idx] += 1
    
print("(indice_Bigrama, indice_proxima_palavra) = qtd_ocorrencias")
print(n_grams_frequency_matrix)

(indice_Bigrama, indice_proxima_palavra) = qtd_ocorrencias
  (4119, 6761)	7.0
  (6761, 6187)	1.0
  (6187, 3922)	1.0
  (3922, 4721)	1.0
  (4721, 7213)	5.0
  (7213, 4525)	1.0
  (4525, 4946)	4.0
  (4946, 8771)	1.0
  (8771, 4156)	1.0
  (4156, 3698)	1.0
  (3698, 517)	7.0
  (517, 4704)	1.0
  (4704, 6487)	66.0
  (6487, 479)	2.0
  (479, 1341)	9.0
  (1341, 3844)	1.0
  (3844, 6321)	1.0
  (6321, 5855)	1.0
  (5855, 3698)	4.0
  (3698, 6718)	2.0
  (6718, 6695)	1.0
  (6695, 6924)	4.0
  (6924, 2515)	1.0
  (2515, 4721)	1.0
  (4721, 3668)	3.0
  :	:
  (5675, 5775)	1.0
  (5775, 6820)	1.0
  (626, 8014)	1.0
  (8014, 4308)	1.0
  (4308, 7439)	1.0
  (4284, 4877)	1.0
  (4877, 7092)	1.0
  (108, 1411)	1.0
  (1411, 381)	1.0
  (381, 494)	1.0
  (494, 2795)	1.0
  (2795, 3738)	1.0
  (494, 3282)	1.0
  (3282, 700)	1.0
  (4557, 6487)	1.0
  (6127, 6697)	1.0
  (6697, 4728)	1.0
  (4728, 6251)	1.0
  (1836, 8515)	1.0
  (8515, 7737)	1.0
  (6741, 7747)	1.0
  (7747, 2738)	1.0
  (2738, 8263)	1.0
  (8263, 4538)	1.0
  (416, 7828)	1

### Mudar forma que o weighted choice funciona
Atualmente o que ele faz é pegar o primeiro item do vetor weights_array, que é o vetor com a probabilidade de cada palavra aparecer 

In [64]:
# EXCLUIR ESSA AQUI
weights_array = n_grams_frequency_matrix[n_gram_idx_dict["o engenho"]].toarray().cumsum()
# [0,1/7, 0, 0, 1/7, 0, 0]

sum_of_weights = weights_array.sum() # A soma vai ser sempre igual a 1
# standardization:
np.multiply(weights_array, 1000 / sum_of_weights, weights_array)
weights_array = weights_array.cumsum()
weights_array[2200:2300]

KeyError: 'o engenho'

In [86]:
def weighted_choice(weights_array, distinct_words):
    """ 
    Função que retorna um elemento aleatório da sequência de 'objetos',
    a probabilidade dos objetos é ponderada de acordo com a porcentagem.
    """
    sum_of_weights = weights_array.sum() # A soma vai ser sempre igual a 1
    # standardization:
    np.multiply(weights_array, 1000 / sum_of_weights, weights_array)
    weights_array = weights_array.cumsum()
    print("Pesos acumulados", weights_array)
    x = random()
    print("Número aleatório", x)

    for i in range(len(weights_array)): # len(weights_array) = len(distinct_words)
        if x < weights_array[i]: # Se o número aleatório for menor que a probabilidade acumulada
            print("i = ", i)
            return distinct_words[i]
        
def get_next_word(word_sequence):
    """
    Função que retorna a próxima palavra após a sequência de palavras
    """
    next_word_vector = n_grams_frequency_matrix[n_gram_idx_dict[word_sequence]] # Linha do bigrama
    next_word_probability = next_word_vector/next_word_vector.sum()

    next_word = weighted_choice(next_word_probability.toarray(), distinct_words)
    
    print("Probabilidade da próxima palavra" , next_word_probability.toarray())
    print("Próxima palavra" , next_word)    
    return next_word
    
def markov_chain(initial_text, chain_length=30):
    """
    Função que gera um texto de acordo com o texto inicial
    """
    current_words = initial_text.split(' ')
    sentence = initial_text

    for _ in range(chain_length):
        sentence += ' '
        next_word = get_next_word(' '.join(current_words))
        sentence += next_word
        current_words = current_words[1:] + [next_word]
    return sentence

In [67]:
markov_chain('dom casmurro capítulo', chain_length=50)

Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.5073788058730968
i =  3922
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra primeiro
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.6260479924977317
i =  4721
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra do
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.8036657673798255
i =  7213
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra título
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.3352641159429398
i =  4525
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra uma
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.8302339544966082
i =  4946
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra noite
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleat

'dom casmurro capítulo primeiro do título uma noite destas vindo da cidade para o engenho novo encontrei no trem da central estourasse longe dos meus ouvidos e interrompesse a linha por muitas horas ainda que morresse alguém e no dia seguinte revelou me o mistério ao primeiro aspecto confesso que fiquei deslumbrado trazia'

In [ ]:
markov_chain('capitu não', chain_length=50)

Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.8090088806065345
i =  1271
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra quero
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.7333091810924155
i =  54
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra isso
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.38289040269553376
i =  8560
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra ouviu?
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.9734468163996607
i =  6187
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra capítulo
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.23800392444179563
i =  2467
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra cxvii
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número a

'capitu não quero isso ouviu? capítulo cxvii amigos próximos já então namorava o piano e começou a interessar se pelos negócios domésticos a cuidar dos passarinhos do pádua quando lhe morreu o marido pedro de albuquerque santiago contava quarenta e dois meses depois ezequiel morreu de uma cruel enfermidade nada menos que'

In [ ]:
markov_chain('agora que', chain_length=50)

Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.16807212816035977
i =  131
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra contei
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.9920152077460637
i =  6321
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra no
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.7305567528917687
i =  4318
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra cap
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.9685306283064984
i =  3449
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra xviii
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número aleatório 0.938490059721129
i =  8859
Probabilidade da próxima palavra [[0. 0. 0. ... 0. 0. 0.]]
Próxima palavra justamente
Pesos acumulados [   0.    0.    0. ... 1000. 1000. 1000.]
Número alea

'agora que contei no cap xviii justamente quando contei isto a aliviasse de cuidados penosos não perdoou à minha casmurrice não sei até se deu um beijo e despedir me não deitar depois do meu amigo venho explicar te que tive tais ciúmes pelo que não desejasse uma queda a algumas adivinhei'

In [87]:
def viterbi(initial_text, n_grams_frequency_matrix, distinct_sets_of_n_words, word_idx_dict, n_grams=1, num_words=100):
    # Quebrar o texto inicial em n-grams
    initial_n_grams = initial_text.split()
    
    # Convertendo a matriz dok para uma matriz densa
    transition_probabilities = n_grams_frequency_matrix.toarray()
    
    # Algoritmo de Viterbi
    sequence = initial_n_grams
    for _ in range(num_words):
        current_state = sequence[-n_grams:]
        current_state_idx = distinct_sets_of_n_words.index(' '.join(current_state))
        #print(current_state_idx)
        
        # Calcular as probabilidades de transição para o próximo estado
        next_state_probs = transition_probabilities[current_state_idx]
        #print(next_state_probs)
        
        # Escolher o próximo estado baseado na probabilidade
        next_state_idx = np.argmax(next_state_probs)
        #print(next_state_idx)
        next_state = distinct_words[next_state_idx].split()[0]
        
        # Adicionar a próxima palavra à sequência
        sequence.append(next_state)
    
    return ' '.join(sequence)

In [88]:
viterbi("dom", n_grams_frequency_matrix, distinct_sets_of_n_words, word_idx_dict, num_words=50)

'dom casmurro não me que não me que não me que não me que não me que não me que não me que não me que não me que não me que não me que não me que não me que não me que não me que não me que não'